In [23]:
import os
import logging
import json
import uuid
from datetime import datetime
import time
import sys
import warnings
import pandas as pd
from tqdm import trange
from dotenv import load_dotenv
from io import BytesIO
import io
import zipfile
import re
from llama_index.legacy import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.core.node_parser import SimpleNodeParser, SentenceWindowNodeParser
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
from llama_index.legacy import Document
from llama_index.core.schema import MetadataMode



# Adobe PDF Services imports
from adobe.pdfservices.operation.auth.service_principal_credentials import ServicePrincipalCredentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.io.cloud_asset import CloudAsset
from adobe.pdfservices.operation.io.stream_asset import StreamAsset
from adobe.pdfservices.operation.pdf_services import PDFServices
from adobe.pdfservices.operation.pdf_services_media_type import PDFServicesMediaType
from adobe.pdfservices.operation.pdfjobs.jobs.extract_pdf_job import ExtractPDFJob
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_pdf_params import ExtractPDFParams
from adobe.pdfservices.operation.pdfjobs.result.extract_pdf_result import ExtractPDFResult

# Pinecone and Langchain imports
from pinecone import Pinecone
from pinecone_text.sparse import BM25Encoder
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')
hf_key = os.getenv('HUGGINGFACE_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
dense_embedder_api = os.getenv("HF_API_URL")

# Initialize clients
pc = Pinecone(api_key=pinecone_api_key)

# Define model
chat_model = "llama3-8b-8192"
index = pc.Index('hsi-notes')
namespace = 'Chapter-1'

In [20]:
# Initialize the logger
logging.basicConfig(level=logging.INFO)

class ExtractTextTableInfoFromPDF:
    def __init__(self, file_path):
        self.unique_id = str(uuid.uuid4())
        try:
            file = open(file_path, 'rb')
            input_stream = file.read()
            file.close()

            # Initial setup, create credentials instance
            credentials = ServicePrincipalCredentials(
                client_id=os.getenv('ADOBE_SERVICES_CLIENT_ID'),
                client_secret=os.getenv('ADOBE_SERVICES_CLIENT_SECRET')
            )

            # Creates a PDF Services instance
            pdf_services = PDFServices(credentials=credentials)

            # Creates an asset(s) from source file(s) and upload
            input_asset = pdf_services.upload(input_stream=input_stream, mime_type=PDFServicesMediaType.PDF)

            # Create parameters for the job
            extract_pdf_params = ExtractPDFParams(
                elements_to_extract=[ExtractElementType.TEXT, ExtractElementType.TABLES],
            )

            # Creates a new job instance
            extract_pdf_job = ExtractPDFJob(input_asset=input_asset, extract_pdf_params=extract_pdf_params)

            # Submit the job and gets the job result
            location = pdf_services.submit(extract_pdf_job)
            pdf_services_response = pdf_services.get_job_result(location, ExtractPDFResult)

            # Get content from the resulting asset(s)
            result_asset: CloudAsset = pdf_services_response.get_result().get_resource()
            stream_asset: StreamAsset = pdf_services.get_content(result_asset)
            
            zip_bytes = io.BytesIO(stream_asset.get_input_stream())
            with zipfile.ZipFile(zip_bytes, 'r') as zip_ref:
                # Extract all the contents into memory
                self.extracted_data = {name: zip_ref.read(name) for name in zip_ref.namelist()}
                
        except (ServiceApiException, ServiceUsageException, SdkException) as e:
            logging.exception(f'Exception encountered while executing operation: {e}')
    
    # Generates a string containing a directory structure and file name for the output file using unique_id
    @staticmethod
    def create_output_file_path(unique_id: str) -> str:
        os.makedirs("../data/Extracted_data", exist_ok=True)
        return f"../data/Extracted_data/{unique_id}.zip"

    @classmethod
    def create_with_unique_id(cls, file_path):
        instance = cls(file_path)
        return instance, instance.unique_id

In [21]:
file_path = '../data/HSI1000-chapter1.pdf'
extractor, unique_id = ExtractTextTableInfoFromPDF.create_with_unique_id(file_path)
print("\nUnique ID:", unique_id)

2024-06-29 00:08:41,343 - INFO - Started uploading asset
2024-06-29 00:08:45,329 - INFO - Finished uploading asset
2024-06-29 00:08:45,333 - INFO - Started submitting EXTRACT_PDF job
2024-06-29 00:08:46,707 - INFO - Started getting job result
2024-06-29 00:08:59,554 - INFO - Finished polling for status
2024-06-29 00:08:59,558 - INFO - Finished getting job result
2024-06-29 00:08:59,559 - INFO - Started getting content
2024-06-29 00:08:59,891 - INFO - Finished getting content


Unique ID: 5f85507f-0e05-4f6c-8c07-c4a1ef621991


In [ ]:
table_output_directory = f"../data/{unique_id}"

# Get the extracted data from the extractor
def get_extracted_data(extracted_data):
    if 'structuredData.json' in extracted_data:
        json_data = json.loads(extracted_data['structuredData.json'])
    return json_data



# This function converts tables to strings to be able to be processed by LLMs. 
def get_and_save_table_strings(extractor, table_output_directory):
    
    # Function to convert each row in a raw unprocessed table (ie index of table not decided) into a string
    def get_raw_table_string(df):
        table_str = ""
        for i in range(2):
            if i ==1:
                table_str += f"Row {i}: {df.iloc[i].values.tolist()}"  
            else:
                table_str += f"Row {i}: {df.iloc[i].values.tolist()}\n"
        return table_str
    
    # Function to decide if header is first row or first column
    def evaluate_table_index_llama(table_str):
        class Header(BaseModel):
            index: int = Field(description="Header of the table, 0 for first row as the header, 1 for first column as the header")
            
        parser = JsonOutputParser(pydantic_object=Header)

        chat = ChatGroq(temperature=0, model_name="llama3-8b-8192")
        template = '''You will assist me in deciding, based on the first 2 entries of a table, whether the first row or the first colum should be the header. 
                You are to output an int, 0 or 1. Where 0 if the first row is header, and 1 if the first column is the header.
                Follow the format instructions carefully.
                Table:
                {table}
                
                {format_instructions}
                '''
        prompt = PromptTemplate(
            template=template,
            input_variables=["table"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )
        chain = prompt | chat | parser
        return chain.invoke({"table": table_str})

    # Tables need procecssing when extraced from BytesIO
    def clean_table_values(x):
        if isinstance(x, str):
            return x.replace('_x000D_', '').strip()
        return x
    
    # Code adapted from a medium blog on how to represent rows of tables in strings
    def convert_table_to_string(df):
        for _, row in df.iterrows():
            row_str = ""
            for col in df.columns:
                sentences = re.split(r'(?<=\.)\s*', row[col])
                row_sentence = ""
                for i in range(len(sentences)):
                    row_sentence += sentences[i] + "\n"
                row_str += f"{col}: {row_sentence}, "
            formatted = row_str[:-2]
        return formatted

    #The literal extraction of the file itself
    os.makedirs(table_output_directory, exist_ok=True)
    excel_files = {k: v for k, v in extractor.extracted_data.items() if k.endswith('.xlsx')}
    table_dataframes = {}

    num_tables =0
    for _, content in excel_files.items():
        excel_stream = BytesIO(content)
        df = pd.read_excel(excel_stream, header=None)
        df = df.applymap(clean_table_values)
        # # Uncomment bottom code to ensure that Groq decides table header index 
        # df_str = get_eval_table_string(df) 
        # dic = evaluate_table_index_llama(df_str)
        # header_index = dic['index']
        header_index = 1
        
        # If header_index is non zero
        if header_index:
            df = pd.read_excel(excel_stream, header=None)
            df = df.applymap(clean_table_values)
            df = df.T
            # Set the first row as the new header
            new_header = df.iloc[0]  # Take the first row for the header
            df = df[1:]  # Take the data less the header row
            df.columns = new_header  # Set the header row as the df header
            # Optionally, reset index if necessary
            df.reset_index(drop=True, inplace=True)
        else:
            df = pd.read_excel(excel_stream, header=0)
        
        output_file_path = os.path.join(table_output_directory, f"table_{num_tables}.xlsx")

        # Writing the tables to csv files. 
        df.to_csv(output_file_path, index=False)
            
        table_str = convert_table_to_string(df)
        table_dataframes[num_tables] = table_str
        num_tables += 1
    return table_dataframes



# Function to initialise a flexible text splitter
def initialise_text_splitter(chunk_size, chunk_overlap):
    separator_list = ["\n\n", "\n", ". ", "!", "?", ",", " ", "", ")", "("]
    try:
        text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                length_function=len,
                separators=separator_list)
    except Exception as e:
        logging.error(f"Failed to initialize text splitter: {e}")
        return None
    return text_splitter



# Function to obtain text chunks using the text splitter
def get_text_chunks(json_data, text_splitter):
    if 'elements' not in json_data:
        logging.error("Missing 'elements' key in json_data")
        raise ValueError("Missing 'elements' key in json_data")

    if not text_splitter:
        logging.error("Text splitter not initialised properly. ")
        sys.exit()  

    # Chunks are split by pages here
    page_text = ""
    start_page = 0
    all_chunks = []
    
    list_label = ""
    for i in range(len(json_data['elements'])):
        try:
            current_page = json_data['elements'][i]['Page']
        except KeyError:
            logging.warning(f"Missing 'Page' key in element at index {i}")
            continue

        try:
            if current_page > start_page:
                # Update the new page number
                start_page = current_page               
                # Generate the chunks for the previous page
                separated_list = text_splitter.split_text(page_text)
                for chunk in separated_list:
                    if chunk not in [". ", "."]:  # Simplified condition
                        all_chunks.append({'ElementType': 'Text', 'Page': current_page, 'Text': chunk})
                # Update the string of text 
                page_text = ""
                list_label = ""
            else:
                if 'Text' in json_data['elements'][i]:  # Check if Text is not empty
                    if json_data['elements'][i]['Path'].endswith("Lbl") and not json_data['elements'][i]['Path'].startswith("//Document/Table"):
                        list_label = json_data['elements'][i]['Text']
                    else:
                        if list_label:
                            page_text += list_label + json_data['elements'][i]['Text']
                            list_label = ""  # Reset list label to empty string
                        else:
                            page_text += json_data['elements'][i]['Text'] + "\n"
        except KeyError as e:
            logging.warning(f"Key error in json_data['elements'][i] processing at index {i}: {e}")
    
    
    # Processing the last page of the text
    if page_text:
        separated_list = text_splitter.split_text(page_text)
        for chunk in separated_list:
            if chunk not in [". ", "."]:
                all_chunks.append({'ElementType': 'Text', 'Page': start_page + 1, 'Text': chunk})

    return all_chunks



# Further enhancement to include the tables with the metadata so that it can be parsed to the 
    # function that takes in the tables + metadata for embeddings generation
def get_table_strings_with_metadata(table_dataframes, json_data):
    
    # Function to obtain the page number of each table
    def get_table_pages(json_data):
        table_file_pages = {}
        # Obtaining the table metadata
        for i in range(len(json_data['elements'])):
            try:
                file_paths = json_data['elements'][i].get('filePaths')
                if file_paths:
                    page = json_data['elements'][i].get('Page', 'Unknown')
                    match = re.search(r'\d+', file_paths[0])
                    table_index = match.group(0)
                    table_file_pages[int(table_index)] = {"Page": page}
            except Exception as e:
                logging.error(f"Error processing file paths at index {i}: {e}")
        return table_file_pages
    
    table_file_pages = get_table_pages(json_data)
    meta_table_batch = []
    table_dfs = []
    for table_index,_ in table_dataframes.items():
        dic = {}
        dic['ElementType'] = 'Table'
        dic['Page'] = table_file_pages[table_index]['Page']
        dic['Table'] = table_dataframes[table_index]
        table_dfs.append(dic)

        #  Obtain metadata for sparse embeddings
        meta_table_batch.append(f"ElementType 'Table', Page {table_file_pages[table_index]['Page']}, {table_dataframes[table_index]}")
    return table_dfs, meta_table_batch

In [33]:
# Here is how we can define our documents for llama index formatting
text_list = ["This is a paragraph in page 1", "This is another paragraph in page 1"]
documents = [Document(text=t,
                      metadata={
                            "file_name": "hsi-chapter-1.pdf",
                            "page": 1
                        },
                      excluded_llm_metadata_keys=["file_name"],
                      metadata_seperator="::",
                      metadata_template="{key}=>{value}",
                      text_template="Metadata: {metadata_str}\n-----\nContent: {content}")
             for t in text_list]
print(
    "The LLM sees this: \n",
    documents[0].get_content(metadata_mode=MetadataMode.LLM),
)
print()
print(
    "The Embedding model sees this: \n",
    documents[0].get_content(metadata_mode=MetadataMode.EMBED),
)

The LLM sees this: 
 Metadata: page=>1
-----
Content: This is a paragraph in page 1

The Embedding model sees this: 
 Metadata: file_name=>hsi-chapter-1.pdf::page=>1
-----
Content: This is a paragraph in page 1


In [ ]:
# Test out the node or window parser